In [13]:
import re
import json
from genericpath import exists
import os
import pandas as pd

In [36]:
nom="sessions"
path_table=os.walk(f"/workspaces/F1_website/data/raw/{nom}")

df=pd.DataFrame()
for fichiers in path_table:
    for fichier in fichiers[2]:
        with open(fichiers[0]+"/"+fichier) as f:
            data = json.load(f)
            df=pd.concat([df,pd.json_normalize(data)])
df = df[["session_key","session_type","session_name","date_start","date_end","meeting_key","year"]]

display(df)

,session_key,session_type,session_name,date_start,date_end,meeting_key,year
0,9462,Practice,Day 1,2024-02-21T07:00:00+00:00,2024-02-21T16:00:00+00:00,1228,2024
1,9463,Practice,Day 2,2024-02-22T07:00:00+00:00,2024-02-22T16:00:00+00:00,1228,2024
2,9464,Practice,Day 3,2024-02-23T07:00:00+00:00,2024-02-23T16:00:00+00:00,1228,2024
3,9465,Practice,Practice 1,2024-02-29T11:30:00+00:00,2024-02-29T12:30:00+00:00,1229,2024
4,9466,Practice,Practice 2,2024-02-29T15:00:00+00:00,2024-02-29T16:00:00+00:00,1229,2024
...,...,...,...,...,...,...,...
117,9832,Practice,Practice 1,2025-12-05T09:30:00+00:00,2025-12-05T10:30:00+00:00,1276,2025
118,9833,Practice,Practice 2,2025-12-05T13:00:00+00:00,2025-12-05T14:00:00+00:00,1276,2025
119,9834,Practice,Practice 3,2025-12-06T10:30:00+00:00,2025-12-06T11:30:00+00:00,1276,2025
120,9835,Qualifying,Qualifying,2025-12-06T14:00:00+00:00,2025-12-06T15:00:00+00:00,1276,2025


In [15]:
display(df.loc[~df["pit_duration"].isna()])

,date,session_key,driver_number,meeting_key,lap_number,pit_duration,lane_duration,stop_duration
9,2024-02-21T07:03:30.378000+00:00,9462,31,1228,2,30.400,30.400,NaN
11,2024-02-21T07:10:28.057000+00:00,9462,77,1228,2,471.200,471.200,NaN
12,2024-02-21T07:10:39.250000+00:00,9462,81,1228,2,29.300,29.300,NaN
13,2024-02-21T07:11:02.263000+00:00,9462,63,1228,7,34.600,34.600,NaN
14,2024-02-21T07:11:21.229000+00:00,9462,23,1228,3,424.200,424.200,NaN
...,...,...,...,...,...,...,...,...
9597,2025-07-06T15:26:39.555000+00:00,9947,16,1277,42,28.840,28.840,2.6
9598,2025-07-06T15:26:43.790000+00:00,9947,23,1277,42,29.200,29.200,NaN
9599,2025-07-06T15:26:45.946000+00:00,9947,31,1277,42,28.646,28.646,2.5
9600,2025-07-06T15:27:27.684000+00:00,9947,81,1277,43,40.280,40.280,14.6


In [56]:
nom="session_result"
with open(f"/workspaces/F1_website/data/raw/{nom}/{nom}.json") as f:
    df=pd.json_normalize(json.load(f))

df = df[["position","driver_number","number_of_laps","dnf","dns","dsq","duration","gap_to_leader","session_key","points"]]
df["number_of_laps"] = df["number_of_laps"].astype('Int64')
df["duration"] = df["duration"].astype(str)
df["gap_to_leader"] = df["gap_to_leader"].astype(str)

#df.to_parquet(f"/workspaces/F1_website/data/staging/{nom}/{nom}.parquet", compression="ZSTD")

display(df) #completer les numero de pilots

,position,driver_number,number_of_laps,dnf,dns,dsq,duration,gap_to_leader,session_key,points
0,1.0,24,133,False,False,False,91.61,0,7763,NaN
1,2.0,1,47,False,False,False,91.65,0.04,7763,NaN
2,3.0,14,130,False,False,False,92.205,0.595,7763,NaN
3,4.0,21,74,False,False,False,92.222,0.612,7763,NaN
4,5.0,27,68,False,False,False,92.466,0.856,7763,NaN
...,...,...,...,...,...,...,...,...,...,...
7111,NaN,12,23,True,False,False,None,None,9947,0.0
7112,NaN,6,17,True,False,False,None,None,9947,0.0
7113,NaN,5,3,True,False,False,None,None,9947,0.0
7114,NaN,30,0,True,False,False,None,None,9947,0.0


In [47]:
table_nation=df[["full_name","country_code"]][~df["country_code"].isnull()].drop_duplicates()

table=df[["meeting_key","driver_number", "full_name","name_acronym","team_name","headshot_url"]].drop_duplicates()

table = pd.merge(table, table_nation, on=["full_name"], how="left")

display(table[table["full_name"]=="Max VERSTAPPEN"].drop_duplicates())

,meeting_key,driver_number,full_name,name_acronym,team_name,headshot_url,country_code
0,1140,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
20,1141,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
40,1142,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
60,1143,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
80,1207,1,Max VERSTAPPEN,VER,Red Bull Racing,https://www.formula1.com/content/dam/fom-websi...,NED
...,...,...,...,...,...,...,...
1486,1273,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED
1507,1274,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED
1528,1275,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED
1549,1276,1,Max VERSTAPPEN,VER,Red Bull Racing,https://media.formula1.com/d_driver_fallback_i...,NED


In [ ]:
display(df[["full_name","country_code"]][df["full_name"]=="Max VERSTAPPEN"].drop_duplicates())

,full_name,country_code
0,Max VERSTAPPEN,NED
4723,Max VERSTAPPEN,None


In [5]:
display(df[["meeting_key","team_name","team_colour"]].drop_duplicates())

,meeting_key,team_name,team_colour
0,1140,Red Bull Racing,3671C6
1,1140,Williams,37BEDD
2,1140,McLaren,F58020
3,1140,Alpine,2293D1
5,1140,Aston Martin,358C75
...,...,...,...
7083,1277,Mercedes,00D7B6
7084,1277,Aston Martin,229971
7085,1277,Ferrari,ED1131
7087,1277,Williams,1868DB
